In [ ]:
import numpy as np
import googlemaps

from golf_router import router
from golf_router.station_repository import StationRepository
from golf_router.golf_course_repository import GolfCourseRepository

# Google Maps API Key
client = googlemaps.Client(key="your API key here")

sta_repo = StationRepository("data/dummy_station.csv")
golf_repo = GolfCourseRepository("data/dummy_golf_course.csv")

In [ ]:
NUM_TRIALS = 10
TOP = 64

golfs = golf_repo.read_all()

top_01_64_counts = np.zeros(TOP)
top_16_79_counts = np.zeros(TOP)
top_31_94_counts = np.zeros(TOP)

for i in range(NUM_TRIALS):
    stas = sta_repo.read_randomly()

    dists_dist = []
    durations_gmaps = []

    for golf in golfs:
        _, dist_dist = router.find_route_by_distance(stas[0], stas[1:], golf)
        dists_dist.append(dist_dist)
        _, duration_gmaps = router.find_route_by_gmaps(client, stas[0], stas[1:], golf)
        durations_gmaps.append(duration_gmaps)

    sorted_dists_dist = np.argsort(dists_dist)
    sorted_duras_gmap = np.argsort(durations_gmaps)

    for i in range(TOP):
        if sorted_duras_gmap[i] in sorted_dists_dist[:TOP]:
            top_01_64_counts[i] += 1
        if sorted_duras_gmap[i+15] in sorted_dists_dist[:TOP+15]:
            top_16_79_counts[i] += 1
        if sorted_duras_gmap[i+30] in sorted_dists_dist[:TOP+30]:
            top_31_94_counts[i] += 1

print(top_01_64_counts / NUM_TRIALS)
print(top_16_79_counts / NUM_TRIALS)
print(top_31_94_counts / NUM_TRIALS)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 1)

axes[0].bar(range(TOP), top_01_64_counts / NUM_TRIALS, edgecolor="white", width=1.0)
axes[1].bar(range(15, TOP+15), top_16_79_counts / NUM_TRIALS, edgecolor="white", width=1.0)
axes[2].bar(range(30, TOP+30), top_31_94_counts / NUM_TRIALS, edgecolor="white", width=1.0)